# Music generation with Deep Learning

This notebook is based on https://www.analyticsvidhya.com/blog/2020/01/how-to-perform-automatic-music-generation/

However this one requires Python 3 and Tensorflow 2.

In [1]:
from numpy.random import seed
from music21 import * 

import numpy as np     
import glob

import importlib
import tensorflow as tf
import keras
import keras.backend as K
import random         

from os import environ
from keras.layers import * 
from keras.models import *

Using TensorFlow backend.


In [2]:
seed(1)
tf.random.set_seed(2)

Defining a function to read the MIDI file.

In [3]:
def read_midi(file):
  notes=[]
  notes_to_parse = None

  #parsing a midi file
  midi = converter.parse(file)
  #grouping based on different instruments
  instruments = instrument.partitionByInstrument(midi)

  #Looping over all the instruments
  for part in instruments.parts:
    #select elements of only piano
    if 'Piano' in str(part): 
      notes_to_parse = part.recurse() 
      #finding whether a particular element is note or a chord
      for element in notes_to_parse:
        if isinstance(element, note.Note):
          notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
          notes.append('.'.join(str(n) for n in element.normalOrder))
      
  return notes

In [4]:
#read all the filenames
path = "./datasets/audio/schubert/"
files = [f for f in glob.glob(path + "*.mid", recursive=False)]

In [5]:
#reading each midi file
all_notes=[]
for i in files:
  all_notes.append(read_midi(i))

#notes and chords of all the midi files
notes = [element for notes in all_notes for element in notes]

Preparing the input and output sequences.

In [6]:
#length of a input sequence
no_of_timesteps = 128      

#no. of unique notes
n_vocab = len(set(notes))  

#all the unique notes
pitch = sorted(set(item for item in notes))  

#assign unique value to every note
note_to_int = dict((note, number) for number, note in enumerate(pitch))  

#preparing input and output sequences
X = []
y = []
for notes in all_notes:
  for i in range(0, len(notes) - no_of_timesteps, 1):
    input_ = notes[i:i + no_of_timesteps]
    output = notes[i + no_of_timesteps]
    X.append([note_to_int[note] for note in input_])
    y.append(note_to_int[output])

The input to the Convolution 1D or LSTM has to be in the form of (samples, timesteps, features).

In [7]:
X = np.reshape(X, (len(X), no_of_timesteps, 1))
#normalizing the inputs
X = X / float(n_vocab)   

Define 2 architectures – WaveNet and LSTM and experiment with them. 

In [8]:
def lstm():
  model = Sequential()
  model.add(LSTM(128,return_sequences=True))
  model.add(LSTM(128))
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  return model

In [9]:
K.clear_session()

def simple_wavenet():
  no_of_kernels=64
  num_of_blocks= int(np.sqrt(no_of_timesteps)) - 1   #no. of stacked conv1d layers

  model = Sequential()
  for i in range(num_of_blocks):
    model.add(Conv1D(no_of_kernels,3,dilation_rate=(2**i),padding='causal',activation='relu'))
    
  model.add(Conv1D(1, 1, activation='relu', padding='causal'))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(n_vocab, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  return model

In [13]:
#define callback to save the model after every 50 epochs.
mc = keras.callbacks.ModelCheckpoint('model{epoch:03d}.h5', save_weights_only=False, period=50)
#Instantiate and train the model with a batch size of 128
model = simple_wavenet()
model.fit(X,np.array(y), epochs=300, batch_size=128,callbacks=[mc])

Epoch 1/300
63757/63757 [==============================] - 245s 4ms/step - loss: 4.8197
Epoch 2/300
63757/63757 [==============================] - 233s 4ms/step - loss: 4.6497
Epoch 3/300
63757/63757 [==============================] - 237s 4ms/step - loss: 4.5983
Epoch 4/300
63757/63757 [==============================] - 237s 4ms/step - loss: 4.5580
Epoch 5/300
63757/63757 [==============================] - 234s 4ms/step - loss: 4.5083
Epoch 6/300
63757/63757 [==============================] - 232s 4ms/step - loss: 4.4533
Epoch 7/300
63757/63757 [==============================] - 245s 4ms/step - loss: 4.3878
Epoch 8/300
63757/63757 [==============================] - 251s 4ms/step - loss: 4.3231
Epoch 9/300
63757/63757 [==============================] - 264s 4ms/step - loss: 4.2601
Epoch 10/300
63757/63757 [==============================] - 251s 4ms/step - loss: 4.1999
Epoch 11/300
63757/63757 [==============================] - 257s 4ms/step - loss: 4.1512
Epoch 12/300
63757/63757 [====

63757/63757 [==============================] - 238s 4ms/step - loss: 2.7799
Epoch 94/300
63757/63757 [==============================] - 229s 4ms/step - loss: 2.7685
Epoch 95/300
63757/63757 [==============================] - 240s 4ms/step - loss: 2.7585
Epoch 96/300
63757/63757 [==============================] - 264s 4ms/step - loss: 2.7637
Epoch 97/300
63757/63757 [==============================] - 248s 4ms/step - loss: 2.7492
Epoch 98/300
63757/63757 [==============================] - 273s 4ms/step - loss: 2.7457
Epoch 99/300
63757/63757 [==============================] - 255s 4ms/step - loss: 2.7373
Epoch 100/300
63757/63757 [==============================] - 239s 4ms/step - loss: 2.7372
Epoch 101/300
63757/63757 [==============================] - 229s 4ms/step - loss: 2.7262
Epoch 102/300
63757/63757 [==============================] - 231s 4ms/step - loss: 2.7226
Epoch 103/300
63757/63757 [==============================] - 241s 4ms/step - loss: 2.7177
Epoch 104/300
63757/63757 [===

63757/63757 [==============================] - 244s 4ms/step - loss: 2.3887
Epoch 185/300
63757/63757 [==============================] - 244s 4ms/step - loss: 2.3991
Epoch 186/300
63757/63757 [==============================] - 269s 4ms/step - loss: 2.4008
Epoch 187/300
63757/63757 [==============================] - 248s 4ms/step - loss: 2.3944
Epoch 188/300
63757/63757 [==============================] - 235s 4ms/step - loss: 2.3874
Epoch 189/300
63757/63757 [==============================] - 231s 4ms/step - loss: 2.3898
Epoch 190/300
63757/63757 [==============================] - 228s 4ms/step - loss: 2.3821
Epoch 191/300
63757/63757 [==============================] - 230s 4ms/step - loss: 2.3811
Epoch 192/300
63757/63757 [==============================] - 239s 4ms/step - loss: 2.3745
Epoch 193/300
63757/63757 [==============================] - 235s 4ms/step - loss: 2.3716
Epoch 194/300
63757/63757 [==============================] - 239s 4ms/step - loss: 2.3682
Epoch 195/300
63757/6375

63757/63757 [==============================] - 246s 4ms/step - loss: 2.2044
Epoch 276/300
63757/63757 [==============================] - 244s 4ms/step - loss: 2.1980
Epoch 277/300
63757/63757 [==============================] - 237s 4ms/step - loss: 2.2102
Epoch 278/300
63757/63757 [==============================] - 221s 3ms/step - loss: 2.2010
Epoch 279/300
63757/63757 [==============================] - 222s 3ms/step - loss: 2.2040
Epoch 280/300
63757/63757 [==============================] - 234s 4ms/step - loss: 2.1953
Epoch 281/300
63757/63757 [==============================] - 249s 4ms/step - loss: 2.1934
Epoch 282/300
63757/63757 [==============================] - 242s 4ms/step - loss: 2.1995
Epoch 283/300
63757/63757 [==============================] - 280s 4ms/step - loss: 2.1970
Epoch 284/300
63757/63757 [==============================] - 257s 4ms/step - loss: 2.1893
Epoch 285/300
63757/63757 [==============================] - 253s 4ms/step - loss: 2.2059
Epoch 286/300
63757/6375

In [14]:
#predicts the most likely element for a certain number of iterations
def generate_music(model, pitch, no_of_timesteps, pattern):
    
    int_to_note = dict((number, note) for number, note in enumerate(pitch))
    prediction_output = []
    
    # generate 50 elements
    for note_index in range(50):
        #reshaping array to feed into model
        input_ = np.reshape(pattern, (1, len(pattern), 1))
        #predict the probability and choose the maximum value
        proba = model.predict(input_, verbose=0)
        index = np.argmax(proba)
        #convert integer back to the element
        pred = int_to_note[index]
        prediction_output.append(pred)
        pattern = list(pattern)
        pattern.append(index/float(n_vocab))
        #leave the first value at index 0
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [18]:
#convert the output to MIDI
def convert_to_midi(prediction_output):
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Specify duration between 2 notes
        offset += 0.5
       # offset += random.uniform(0.5,0.9)

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='datasets/audio/out.mid')

In [19]:
#compose our music
#Select random chunk for the first iteration
start = np.random.randint(0, len(X)-1)
pattern = X[start]
#load the best model
model=load_model('model300.h5')
#generate and save music
music = generate_music(model,pitch,no_of_timesteps,pattern)
convert_to_midi(music)